In [1]:
from dataclasses import dataclass,field
from typing import Any, Dict, Sequence, Tuple, Union

In [2]:
@dataclass
class DataClassCard:
    rank: str
    suit: str
    number: Union[int, str]

queen_of_hearts = DataClassCard('Q', 'Hearts', [9,5])

In [3]:
queen_of_hearts.number

[9, 5]

In [4]:
@dataclass
class Hyperparameter:
    learning_rate: float
    batch_size: int
    epochs: int
    optimizer: str

hyperparameter = Hyperparameter(0.001, 32, 10, 'Adam')

In [5]:
hyperparameter.learning_rate

0.001

In [6]:
hyperparameter.__repr__()

"Hyperparameter(learning_rate=0.001, batch_size=32, epochs=10, optimizer='Adam')"

In [7]:
@dataclass
class _Hyperparameter():

    # STFT hyperparameters
    sr: int = 41000
    n_fft: int = 1024
    hop_length: int = 512
    win_length: int = 1024
    window: str = 'hann'
    center: bool = True
    return_complex: bool = True

    # Training hyperparameters
    learning_rate: float = 4e-5
    batch_size: int = 32
    epochs: int = 200
    optimizer: str = 'Adam'

    # paths
    data_path: str = field(init=False)
    model_path: str =  field(init=False)
    log_path: str =  field(init=False)

    def __post_init__(self, data_path, model_path, log_path):
        self.data_path = data_path
        self.model_path = model_path
        self.log_path = log_path



In [8]:
ALL = slice(None)
LAST = slice(-1, None)
HALF = slice(None, None, 2)
NONE = None

In [9]:
l = [1,2,3,4,5,6]
l[HALF]

[1, 3, 5]

In [10]:
import torch
import torch.nn as nn
import torch.functional as F

t = torch.randn(4,4,3,3)

t[1,1,...]

tensor([[-0.1437,  1.6431, -0.2762],
        [-1.2818,  0.2748,  0.6625],
        [-0.9958, -0.3862, -0.2489]])

In [11]:
class GatedLinearUnit:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.fc = nn.Linear(input_size, output_size*2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        return x[:,:self.output_size] * self.sigmoid(x[:,self.output_size:])
    

class GatedConvolutionalUnit:
    def __init__(self, input_size, output_size, kernel_size, stride, padding):
        self.input_size = input_size
        self.output_size = output_size
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.conv = nn.Conv2d(input_size, output_size*2, kernel_size, stride, padding,bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv(x)
        return x[:,:self.output_size] * self.sigmoid(x[:,self.output_size:])

In [12]:
gatedLU = GatedLinearUnit(10, 5)
gatedLU.forward(torch.randn(4,10))



tensor([[ 0.5195,  0.3527, -0.2257,  0.3120,  0.2144],
        [ 0.0636, -0.3561,  0.5108,  0.1135, -0.6693],
        [ 0.0108,  0.1771,  0.1743, -0.0271, -0.0987],
        [ 0.5395, -0.8714,  0.3512, -0.2565, -0.2042]], grad_fn=<MulBackward0>)

In [13]:
convLU = GatedConvolutionalUnit(3, 64, 3, 1, 1)

x = torch.randn(4,3,32,32,dtype=torch.complex64)

convLU.forward(x).shape

RuntimeError: Input type (CPUComplexFloatType) and weight type (torch.FloatTensor) should be the same

In [ ]:
import numpy as np

# Create a complex-valued array
complex_array = np.array([1+2j, 2+3j, 3+4j], dtype=np.complex64)
print(f"Original complex array:\n {complex_array}")

# Convert to float32 and reshape
complex_array_float32 = complex_array.view(dtype=np.float32).reshape((*complex_array.shape, 2))
print(f"\nComplex array as float32:\n {complex_array_float32}")

print(complex_array_float32.shape)

# The first channel is the real part, the second channel is the imaginary part
real_part = complex_array_float32[..., 0]
imaginary_part = complex_array_float32[..., 1]
print(f"\nReal part: {real_part}\nImaginary part: {imaginary_part}")

# Recombine the real and imaginary parts to restore the complex array
restored_complex_array = complex_array_float32.view(dtype=np.complex64).reshape(complex_array.shape)
print(f"\nRestored complex array:\n {restored_complex_array}")


Original complex array:
 [1.+2.j 2.+3.j 3.+4.j]

Complex array as float32:
 [[1. 2.]
 [2. 3.]
 [3. 4.]]
(3, 2)

Real part: [1. 2. 3.]
Imaginary part: [2. 3. 4.]

Restored complex array:
 [1.+2.j 2.+3.j 3.+4.j]


In [ ]:
complex_tensor = torch.randn(4,1,32,32,dtype=torch.complex64)
# torch.randn(4,3,32,32,dtype=torch.complex64).view(dtype=torch.float32).shape



complex_tensor.view(dtype=torch.float32).shape


# real_part = complex_tensor[..., 0]
# imaginary_part = complex_tensor[..., 1]

# print(real_part)

torch.Size([4, 1, 32, 64])

In [ ]:
import torch

# Assuming your tensor `complex_tensor` is of shape (B, C, H, W) and dtype=torch.complex64

complex_tensor = torch.randn((8, 3, 64, 64), dtype=torch.complex64)  # for example

# Split the complex tensor into real and imaginary parts
real_part = complex_tensor.real
imag_part = complex_tensor.imag

# Stack the real and imaginary parts along the channel dimension
stacked_tensor = torch.cat((real_part, imag_part), dim=1)

print(stacked_tensor.shape)  # Will output: torch.Size([8, 6, 64, 64])

real_part = stacked_tensor[:, :3, ...]
imag_part = stacked_tensor[:, 3:, ...]


# Recombine the real and imaginary parts to restore the complex tensor
restored_complex_tensor = real_part + 1j * imag_part

print(restored_complex_tensor.shape)  # Will output: torch.Size([8, 3, 64, 64])

torch.Size([8, 6, 64, 64])
torch.Size([8, 3, 64, 64])


In [15]:
from src.data_loader import *
from torch.utils.data import DataLoader

%reload_ext autoreload
%autoreload 2

if False:

    path = '../data/AudioMNIST_Indicies/dummy_labels.csv'
    root_dir = '../data/AudioMNIST/'

else:
    path = '../data/AvianID_AcousticIndices/UK_AI.csv'
    root_dir='../data/UK_BIRD/'


ds = AvianNatureSounds(annotation_file_path=path,
                       root_dir=root_dir,
                       key='habitat',
                       mode='stft',
                       length=5,
                       sampling_rate=44100,
                       n_fft=1024,
                       hop_length=512,
                       mel_spectrogram=None,
                       verbose=False,
                       fixed_limit=True)



train_loader = DataLoader(dataset=ds, batch_size=4, shuffle=True)

batch = next(iter(train_loader))

batch[0].shape

torch.Size([4, 2, 512, 512])